In [ ]:
from utils.MockData import create_data
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from xgboost import XGBRanker


In [ ]:
le_ = LabelEncoder()
def get_integer_mapping(le):
    '''
    Return a dict mapping labels to their integer values
    from an SKlearn LabelEncoder
    le = a fitted SKlearn LabelEncoder
    '''
    res = {}
    for cl in le.classes_:
        res.update({cl:le.transform([cl])[0]})

    return res

In [ ]:
df = create_data(inadimplencia_rate=0.3)
df['data_vencimento'] = df['data_vencimento'].dt.floor('D')
df['data_pagamento'] = df['data_pagamento'].dt.floor('D')
df['atraso'] = df['data_pagamento'] - df['data_vencimento']
df['atraso'] = df['atraso'].apply(lambda x: x.days if x.days > 0 else 0)
df["sexo_encoded"] = le_.fit_transform(df['sexo'])
df.drop(columns=[
    'sexo',
    "data_vencimento",
    "data_pagamento"
    ], inplace=True)
print("Class 'sexo' mapping:")
get_integer_mapping(le_)

In [ ]:
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop( columns = ['chance_inadimplencia', "inadimplente"]), df['chance_inadimplencia'], test_size=0.33, random_state=42)

In [ ]:
xgb_rkr = XGBRanker(objective='rank:ndcg', n_estimators=100, random_state=0,learning_rate=0.1)